# Convert ECB bronze_fxdata into silver layer

###### Create temporary views for currency and exchange rate with necessary transformation logic

In [2]:
CREATE VIEW IF NOT EXISTS vwcurrency
AS
SELECT 1 as col;

ALTER VIEW vwcurrency
AS
SELECT DISTINCT Currency AS `currency_id` FROM ECB.bronze_fxdata
UNION
SELECT 'EUR' AS `currency_id`;


StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 42 rows and 1 fields>

In [3]:
CREATE VIEW IF NOT EXISTS vwexchangerate
AS
SELECT 1 AS col;

ALTER VIEW vwexchangerate
AS
SELECT 'EUR' AS `base_currency_id`, -- EUR is always the base currency from ECB dataset
Currency AS `quote_currency_id`, -- Currency field is always the quote currency
Date AS  `exchange_date`,
Rate AS `exchange_rate`
FROM ECB.bronze_fxdata;


StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 10 rows and 4 fields>

###### Create silver tables if not exists

In [13]:
-- create currency table
CREATE TABLE IF NOT EXISTS ECB.silver_currency
(
    currency_id string,
    currency_name string,
    create_dt timestamp,
    modify_dt timestamp
) USING delta;

-- create exchange_rate table
CREATE TABLE IF NOT EXISTS ECB.silver_exchange_rate
(
    base_currency_id string,
    quote_currency_id string,
    exchange_date date,
    exchange_rate double,
    create_dt timestamp,
    modify_dt timestamp
) USING delta; 

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

###### load data into currency table

In [15]:
MERGE INTO ECB.silver_currency TGT
USING vwcurrency SRC
ON TGT.currency_id = SRC.currency_id
WHEN NOT MATCHED THEN
INSERT (currency_id,currency_name,create_dt,modify_dt)
VALUES (SRC.currency_id,SRC.currency_id,CURRENT_TIMESTAMP,CURRENT_TIMESTAMP);

StatementMeta(, 65fe56a1-d5aa-43d1-9f10-7fa0b40d9087, 24, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

###### load data into exchange rate table

In [16]:
MERGE INTO ECB.silver_exchange_rate TGT
USING vwexchangerate SRC
ON (TGT.base_currency_id = SRC.base_currency_id AND TGT.quote_currency_id = SRC.quote_currency_id AND TGT.exchange_date = CAST(SRC.exchange_date AS DATE))
WHEN MATCHED AND TGT.exchange_rate <> SRC.exchange_rate THEN
    UPDATE SET TGT.exchange_rate = SRC.exchange_rate,
    TGT.modify_dt = CURRENT_TIMESTAMP
WHEN NOT MATCHED THEN
    INSERT (base_currency_id, quote_currency_id, exchange_date, exchange_rate, create_dt, modify_dt) 
    VALUES (SRC.base_currency_id, SRC.quote_currency_id, SRC.exchange_date, SRC.exchange_rate, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);

StatementMeta(, 65fe56a1-d5aa-43d1-9f10-7fa0b40d9087, 25, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

###### Quick result validation

In [17]:
SELECT * FROM ECB.silver_currency;
SELECT * FROM ECB.silver_exchange_rate WHERE exchange_date = '2024-01-02' AND quote_currency_id IN ('USD','HKD');

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 42 rows and 4 fields>

<Spark SQL result set with 2 rows and 6 fields>